# Phase 4: Advanced Tuning and Final Training

## Introduction

This notebook represents the final phase of the DMML binary risk classification project. Building upon the comprehensive model evaluation from Phase 3, this phase focuses on:

1. **Advanced Hyperparameter Optimization**: Sophisticated tuning strategies using Bayesian optimization
2. **Model Interpretability Analysis**: Understanding feature importance and model decisions
3. **Threshold Optimization**: Fine-tuning classification thresholds for optimal performance
4. **Production Model Training**: Final model training with optimized parameters
5. **Comprehensive Model Validation**: Final validation and performance assessment

**Dependencies:**
- Artifacts from Phase 2 (Preprocessing) and Phase 3 (Modeling)
- Best model identified from Phase 3 model selection
- Custom transformers from `custom_transformers.py`

**Objectives:**
- Achieve optimal model performance through advanced tuning
- Ensure model interpretability and explainability
- Prepare production-ready model with comprehensive validation
- Generate final performance reports and model documentation

# Setup and Initialization

This section handles the initial setup, imports, and loading of artifacts from previous phases.

## Optional: Google Colab Setup

Uncomment and run this cell if working in Google Colab environment.

In [ ]:
# Run on Google Colab (optional)
#from google.colab import drive
#drive.mount('/drive', force_remount=True)

## Import Libraries

Import all necessary libraries for advanced tuning, model interpretation, and final training.

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import warnings
import joblib
import json
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime
from typing import Dict, Any, List, Tuple, Optional

# Core ML libraries
from sklearn.model_selection import (
    StratifiedKFold, cross_validate, TimeSeriesSplit,
    validation_curve, learning_curve
)
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    make_scorer, f1_score, accuracy_score, precision_score, recall_score,
    confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve,
    average_precision_score, precision_recall_curve, matthews_corrcoef,
    classification_report
)
from sklearn.base import clone

# Model interpretation libraries
try:
    import shap
    SHAP_AVAILABLE = True
    print("✓ SHAP available for model interpretation")
except ImportError:
    SHAP_AVAILABLE = False
    print("⚠ SHAP not available. Install with: pip install shap")

try:
    from sklearn.inspection import permutation_importance
    PERMUTATION_IMPORTANCE_AVAILABLE = True
except ImportError:
    PERMUTATION_IMPORTANCE_AVAILABLE = False

# Advanced optimization libraries
try:
    from skopt import BayesSearchCV
    from skopt.space import Real, Integer, Categorical
    BAYESIAN_OPT_AVAILABLE = True
    print("✓ Scikit-optimize available for Bayesian optimization")
except ImportError:
    BAYESIAN_OPT_AVAILABLE = False
    print("⚠ Scikit-optimize not available. Install with: pip install scikit-optimize")

# Additional ML libraries
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

# Set up paths and environment
project_path = os.getcwd()
os.chdir(project_path)
sys.path.append(project_path)

# Import custom transformers
try:
    from Utilities.custom_transformers import (
        STKDEAndRiskLabelTransformer, TargetEngineeringPipeline, BinarizeSinCosTransformer
    )
    print("✓ Custom transformers imported successfully")
except ImportError as e:
    print(f"❌ Error importing custom transformers: {e}")
    raise

# Set random seeds and display settings
np.random.seed(42)
warnings.filterwarnings('ignore', category=FutureWarning)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ All libraries imported successfully")

## Define Paths and Directories

Set up directory structure for loading previous results and saving final outputs.

In [ ]:
# Define base directories
base_dir = os.getcwd()
preprocessing_dir = os.path.join(base_dir, "Classification (Preprocessing)")
modeling_dir = os.path.join(base_dir, "Classification (Modeling)")
final_results_dir = os.path.join(base_dir, "Classification (Final)")

# Create final results directory
os.makedirs(final_results_dir, exist_ok=True)

print(f"📁 Directory Structure:")
print(f"   Preprocessing artifacts: {preprocessing_dir}")
print(f"   Modeling results: {modeling_dir}")
print(f"   Final outputs: {final_results_dir}")

# Verify required directories exist
required_dirs = [preprocessing_dir, modeling_dir]
missing_dirs = [d for d in required_dirs if not os.path.exists(d)]

if missing_dirs:
    print(f"❌ Missing required directories: {missing_dirs}")
    print("Please ensure Phases 2 and 3 have been completed successfully.")
    raise FileNotFoundError(f"Required directories not found: {missing_dirs}")
else:
    print("✓ All required directories found")

## Load Previous Phase Artifacts

Load data, models, and results from previous phases to continue with advanced tuning.

In [ ]:
def load_phase_artifacts(preprocessing_dir: str, modeling_dir: str) -> Dict[str, Any]:
    """
    Load all necessary artifacts from previous phases.
    
    Args:
        preprocessing_dir: Preprocessing artifacts directory
        modeling_dir: Modeling results directory
        
    Returns:
        Dictionary containing loaded artifacts
    """
    print("=== Loading Phase Artifacts ===")
    
    artifacts = {}
    
    # Load preprocessing artifacts
    print("\n📂 Loading preprocessing artifacts...")
    try:
        # Load raw data
        X_train_raw = np.load(os.path.join(preprocessing_dir, 'X_train.npy'), allow_pickle=True)
        X_test_raw = np.load(os.path.join(preprocessing_dir, 'X_test.npy'), allow_pickle=True)
        y_train_raw = np.load(os.path.join(preprocessing_dir, 'y_train.npy'), allow_pickle=True)
        y_test_raw = np.load(os.path.join(preprocessing_dir, 'y_test.npy'), allow_pickle=True)
        
        # Load feature names
        feature_names_file = os.path.join(preprocessing_dir, 'feature_names.joblib')
        if os.path.exists(feature_names_file):
            feature_names = joblib.load(feature_names_file)
        else:
            # Fallback to legacy naming
            feature_names = joblib.load(os.path.join(preprocessing_dir, 'X_feature_names.joblib'))
        
        # Convert to DataFrames
        X_train = pd.DataFrame(X_train_raw, columns=feature_names)
        X_test = pd.DataFrame(X_test_raw, columns=feature_names)
        y_train = pd.Series(y_train_raw, name='DUMMY_TARGET')
        y_test = pd.Series(y_test_raw, name='DUMMY_TARGET')
        
        artifacts.update({
            'X_train': X_train,
            'X_test': X_test,
            'y_train': y_train,
            'y_test': y_test,
            'feature_names': feature_names
        })
        
        print(f"✓ Data loaded: X_train {X_train.shape}, X_test {X_test.shape}")
        
        # Load preprocessors
        preprocessor_files = {
            'preprocessor_general': 'preprocessor_general.joblib',
            'preprocessor_trees': 'preprocessor_trees.joblib'
        }
        
        for name, filename in preprocessor_files.items():
            file_path = os.path.join(preprocessing_dir, filename)
            if os.path.exists(file_path):
                artifacts[name] = joblib.load(file_path)
                print(f"✓ Loaded {name}")
            else:
                # Try alternative naming
                alt_name = filename.replace('_general', '_full')
                alt_path = os.path.join(preprocessing_dir, alt_name)
                if os.path.exists(alt_path):
                    artifacts[name] = joblib.load(alt_path)
                    print(f"✓ Loaded {name} (alternative naming)")
                else:
                    print(f"⚠ {name} not found")
                    artifacts[name] = None
        
        # Load STKDE parameters
        metadata_file = os.path.join(preprocessing_dir, 'preprocessing_metadata.json')
        if os.path.exists(metadata_file):
            with open(metadata_file, 'r') as f:
                metadata = json.load(f)
            
            stkde_params = metadata.get('stkde_parameters', {})
            artifacts['hs_optimal'] = stkde_params.get('hs_optimal', 200.0)
            artifacts['ht_optimal'] = stkde_params.get('ht_optimal', 60.0)
            print(f"✓ STKDE parameters: hs={artifacts['hs_optimal']}, ht={artifacts['ht_optimal']}")
        else:
            artifacts['hs_optimal'] = 200.0
            artifacts['ht_optimal'] = 60.0
            print("⚠ Using default STKDE parameters")
    
    except Exception as e:
        print(f"❌ Error loading preprocessing artifacts: {e}")
        raise
    
    # Load modeling results
    print("\n📊 Loading modeling results...")
    try:
        # Load model selection results
        results_file = os.path.join(modeling_dir, 'model_selection_results.csv')
        if os.path.exists(results_file):
            model_results = pd.read_csv(results_file)
            artifacts['model_results'] = model_results
            print(f"✓ Model results loaded: {len(model_results)} models")
            
            # Identify best model
            if 'f1_mean' in model_results.columns:
                valid_results = model_results.dropna(subset=['f1_mean'])
                if not valid_results.empty:
                    best_model_row = valid_results.loc[valid_results['f1_mean'].idxmax()]
                    artifacts['best_model_name'] = best_model_row['model_name']
                    artifacts['best_model_score'] = best_model_row['f1_mean']
                    print(f"✓ Best model identified: {artifacts['best_model_name']} (F1: {artifacts['best_model_score']:.4f})")
        else:
            print("⚠ Model selection results not found")
            artifacts['model_results'] = pd.DataFrame()
            artifacts['best_model_name'] = None
    
    except Exception as e:
        print(f"❌ Error loading modeling results: {e}")
        artifacts['model_results'] = pd.DataFrame()
        artifacts['best_model_name'] = None
    
    return artifacts

# Load all artifacts
artifacts = load_phase_artifacts(preprocessing_dir, modeling_dir)

# Extract main variables for easy access
X_train = artifacts['X_train']
X_test = artifacts['X_test']
y_train = artifacts['y_train']
y_test = artifacts['y_test']
feature_names = artifacts['feature_names']
hs_optimal = artifacts['hs_optimal']
ht_optimal = artifacts['ht_optimal']
preprocessor_general = artifacts.get('preprocessor_general')
preprocessor_trees = artifacts.get('preprocessor_trees')
model_results = artifacts.get('model_results', pd.DataFrame())
best_model_name = artifacts.get('best_model_name')

print(f"\n✅ Phase artifact loading complete!")
print(f"   Ready for Phase 4 advanced tuning and training")

# Advanced Hyperparameter Optimization

This section implements sophisticated hyperparameter tuning using Bayesian optimization for the best performing model from Phase 3.

## Bayesian Optimization Setup

Configure Bayesian optimization for intelligent hyperparameter search.

In [ ]:
def setup_best_model_pipeline(best_model_name: str, hs_optimal: float, ht_optimal: float,
                             preprocessor_general: Any, preprocessor_trees: Any) -> Tuple[Any, Dict[str, Any], str]:
    """
    Set up the best model pipeline and configuration for advanced tuning.
    
    Args:
        best_model_name: Name of the best model from Phase 3
        hs_optimal, ht_optimal: STKDE parameters
        preprocessor_general, preprocessor_trees: Preprocessing pipelines
        
    Returns:
        Tuple of (model_instance, stkde_config, preprocessing_type)
    """
    print(f"=== Setting up {best_model_name} for Advanced Tuning ===")
    
    # Define STKDE configuration
    stkde_config = {
        'hs': hs_optimal,
        'ht': ht_optimal,
        'strategy': 'quantile',
        'n_classes': 2,
        'intensity_col_name': 'stkde_intensity',
        'label_col_name': 'RISK_LEVEL',
        'n_jobs': -1,
        'random_state': 42,
        'year_col': 'YEAR',
        'month_col': 'MONTH',
        'day_col': 'DAY',
        'hour_col': 'HOUR',
        'lat_col': 'Latitude',
        'lon_col': 'Longitude'
    }
    
    # Import and instantiate the best model
    model_configs = {
        'LogisticRegression': {
            'class': 'sklearn.linear_model.LogisticRegression',
            'params': {'random_state': 42, 'max_iter': 1000, 'class_weight': 'balanced'},
            'preprocessing': 'general',
            'uses_smote': True
        },
        'RandomForest': {
            'class': 'sklearn.ensemble.RandomForestClassifier',
            'params': {'random_state': 42, 'class_weight': 'balanced', 'n_jobs': -1},
            'preprocessing': 'tree',
            'uses_smote': False
        },
        'GradientBoosting': {
            'class': 'sklearn.ensemble.GradientBoostingClassifier',
            'params': {'random_state': 42},
            'preprocessing': 'tree',
            'uses_smote': False
        },
        'XGBoost': {
            'class': 'xgboost.XGBClassifier',
            'params': {'random_state': 42, 'use_label_encoder': False, 'eval_metric': 'logloss', 'n_jobs': -1},
            'preprocessing': 'tree',
            'uses_smote': False
        },
        'LightGBM': {
            'class': 'lightgbm.LGBMClassifier',
            'params': {'random_state': 42, 'n_jobs': -1, 'verbose': -1},
            'preprocessing': 'tree',
            'uses_smote': False
        }
    }
    
    if best_model_name not in model_configs:
        raise ValueError(f"Model {best_model_name} not supported for advanced tuning")
    
    config = model_configs[best_model_name]
    
    # Import and instantiate the model
    module_name, class_name = config['class'].rsplit('.', 1)
    
    if module_name == 'sklearn.linear_model':
        from sklearn.linear_model import LogisticRegression
        model_instance = LogisticRegression(**config['params'])
    elif module_name == 'sklearn.ensemble':
        if class_name == 'RandomForestClassifier':
            from sklearn.ensemble import RandomForestClassifier
            model_instance = RandomForestClassifier(**config['params'])
        elif class_name == 'GradientBoostingClassifier':
            from sklearn.ensemble import GradientBoostingClassifier
            model_instance = GradientBoostingClassifier(**config['params'])
    elif module_name == 'xgboost':
        try:
            from xgboost import XGBClassifier
            model_instance = XGBClassifier(**config['params'])
        except ImportError:
            raise ImportError("XGBoost not available. Please install with: pip install xgboost")
    elif module_name == 'lightgbm':
        try:
            from lightgbm import LGBMClassifier
            model_instance = LGBMClassifier(**config['params'])
        except ImportError:
            raise ImportError("LightGBM not available. Please install with: pip install lightgbm")
    else:
        raise ValueError(f"Unsupported model class: {config['class']}")
    
    # Select appropriate preprocessor
    preprocessing_type = config['preprocessing']
    if preprocessing_type == 'tree' and preprocessor_trees is not None:
        selected_preprocessor = preprocessor_trees
    elif preprocessing_type == 'general' and preprocessor_general is not None:
        selected_preprocessor = preprocessor_general
    else:
        # Fallback to available preprocessor
        selected_preprocessor = preprocessor_general or preprocessor_trees
        if selected_preprocessor is None:
            raise ValueError("No preprocessor available")
    
    print(f"✓ Model: {best_model_name}")
    print(f"✓ Preprocessing: {preprocessing_type}")
    print(f"✓ Uses SMOTE: {config['uses_smote']}")
    
    return model_instance, stkde_config, selected_preprocessor, config['uses_smote']

def define_bayesian_search_space(model_name: str) -> Dict[str, Any]:
    """
    Define search space for Bayesian optimization.
    
    Args:
        model_name: Name of the model
        
    Returns:
        Dictionary defining the search space
    """
    if not BAYESIAN_OPT_AVAILABLE:
        print("⚠ Bayesian optimization not available, falling back to grid search")
        return None
    
    search_spaces = {
        'LogisticRegression': {
            'feature_pipeline__classifier__C': Real(0.001, 100.0, prior='log-uniform'),
            'feature_pipeline__classifier__penalty': Categorical(['l1', 'l2']),
            'feature_pipeline__classifier__solver': Categorical(['liblinear', 'saga'])
        },
        
        'RandomForest': {
            'feature_pipeline__classifier__n_estimators': Integer(50, 500),
            'feature_pipeline__classifier__max_depth': Integer(3, 30),
            'feature_pipeline__classifier__min_samples_split': Integer(2, 20),
            'feature_pipeline__classifier__min_samples_leaf': Integer(1, 10),
            'feature_pipeline__classifier__max_features': Categorical(['sqrt', 'log2', None])
        },
        
        'GradientBoosting': {
            'feature_pipeline__classifier__n_estimators': Integer(50, 500),
            'feature_pipeline__classifier__max_depth': Integer(3, 15),
            'feature_pipeline__classifier__learning_rate': Real(0.001, 0.5, prior='log-uniform'),
            'feature_pipeline__classifier__min_samples_split': Integer(2, 20),
            'feature_pipeline__classifier__min_samples_leaf': Integer(1, 10)
        },
        
        'XGBoost': {
            'feature_pipeline__classifier__n_estimators': Integer(50, 500),
            'feature_pipeline__classifier__max_depth': Integer(3, 15),
            'feature_pipeline__classifier__learning_rate': Real(0.001, 0.5, prior='log-uniform'),
            'feature_pipeline__classifier__min_child_weight': Integer(1, 10),
            'feature_pipeline__classifier__subsample': Real(0.6, 1.0),
            'feature_pipeline__classifier__colsample_bytree': Real(0.6, 1.0)
        },
        
        'LightGBM': {
            'feature_pipeline__classifier__n_estimators': Integer(50, 500),
            'feature_pipeline__classifier__max_depth': Integer(3, 15),
            'feature_pipeline__classifier__learning_rate': Real(0.001, 0.5, prior='log-uniform'),
            'feature_pipeline__classifier__num_leaves': Integer(10, 300),
            'feature_pipeline__classifier__min_child_samples': Integer(5, 100),
            'feature_pipeline__classifier__subsample': Real(0.6, 1.0)
        }
    }
    
    return search_spaces.get(model_name, {})

# Setup best model for advanced tuning
if best_model_name:
    try:
        model_instance, stkde_config, selected_preprocessor, uses_smote = setup_best_model_pipeline(
            best_model_name, hs_optimal, ht_optimal, 
            preprocessor_general, preprocessor_trees
        )
        
        # Define search space
        search_space = define_bayesian_search_space(best_model_name)
        
        print(f"\n✅ Advanced tuning setup complete for {best_model_name}")
        if search_space:
            print(f"   Search space defined with {len(search_space)} parameters")
        
    except Exception as e:
        print(f"❌ Error setting up {best_model_name}: {e}")
        model_instance = None
        search_space = None
else:
    print("⚠ No best model identified from Phase 3")
    model_instance = None
    search_space = None

## Execute Bayesian Optimization

Perform advanced hyperparameter optimization using Bayesian methods.

In [ ]:
def create_advanced_pipeline(model_instance: Any, preprocessor: Any, 
                           stkde_config: Dict[str, Any], use_smote: bool = False) -> Pipeline:
    """
    Create the complete pipeline for advanced optimization.
    
    Args:
        model_instance: The model to optimize
        preprocessor: Preprocessing pipeline
        stkde_config: STKDE configuration
        use_smote: Whether to include SMOTE
        
    Returns:
        Complete pipeline ready for optimization
    """
    # Create STKDE transformer
    stkde_transformer = STKDEAndRiskLabelTransformer(**stkde_config)
    
    # Create feature pipeline steps
    pipeline_steps = [('preprocessor', preprocessor)]
    
    if use_smote:
        pipeline_steps.append(('smote', SMOTE(random_state=42)))
    
    pipeline_steps.append(('classifier', model_instance))
    
    # Use appropriate pipeline class
    if use_smote:
        feature_classifier_pipeline = ImbPipeline(pipeline_steps)
    else:
        feature_classifier_pipeline = Pipeline(pipeline_steps)
    
    # Wrap in target engineering pipeline
    full_pipeline = TargetEngineeringPipeline(
        target_engineer=stkde_transformer,
        feature_pipeline=feature_classifier_pipeline
    )
    
    return full_pipeline

def perform_bayesian_optimization(pipeline: Pipeline, search_space: Dict[str, Any],
                                X_train: pd.DataFrame, y_train: pd.Series,
                                model_name: str, results_dir: str) -> Tuple[Pipeline, Dict[str, Any]]:
    """
    Perform Bayesian hyperparameter optimization.
    
    Args:
        pipeline: Complete model pipeline
        search_space: Bayesian search space
        X_train, y_train: Training data
        model_name: Name of the model
        results_dir: Directory for saving results
        
    Returns:
        Tuple of (optimized_pipeline, optimization_results)
    """
    print(f"\n=== Bayesian Optimization for {model_name} ===")
    
    if not BAYESIAN_OPT_AVAILABLE or not search_space:
        print("⚠ Bayesian optimization not available, skipping")
        return pipeline, {}
    
    try:
        # Sort training data temporally
        temporal_cols = ['YEAR', 'MONTH', 'DAY', 'HOUR']
        if all(col in X_train.columns for col in temporal_cols):
            X_train_sorted = X_train.sort_values(temporal_cols).copy()
            y_train_sorted = y_train.iloc[X_train_sorted.index].copy()
            X_train_sorted.reset_index(drop=True, inplace=True)
            y_train_sorted.reset_index(drop=True, inplace=True)
            cv_strategy = TimeSeriesSplit(n_splits=3)  # Reduced for efficiency
        else:
            X_train_sorted = X_train.copy()
            y_train_sorted = y_train.copy()
            cv_strategy = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        
        print(f"Using {type(cv_strategy).__name__} with {cv_strategy.n_splits} folds")
        
        # Create Bayesian search
        bayesian_search = BayesSearchCV(
            estimator=pipeline,
            search_spaces=search_space,
            n_iter=30,  # Number of optimization iterations
            cv=cv_strategy,
            scoring='f1',
            n_jobs=1,  # Avoid nested parallelization
            random_state=42,
            verbose=1
        )
        
        print(f"Starting Bayesian optimization with {len(search_space)} parameters...")
        start_time = time.time()
        
        # Perform optimization
        bayesian_search.fit(X_train_sorted, y_train_sorted)
        
        optimization_time = time.time() - start_time
        
        print(f"✅ Optimization completed in {optimization_time:.1f} seconds")
        print(f"   Best F1 score: {bayesian_search.best_score_:.4f}")
        print(f"   Best parameters: {bayesian_search.best_params_}")
        
        # Save optimization results
        optimization_results = {
            'best_score': float(bayesian_search.best_score_),
            'best_params': bayesian_search.best_params_,
            'optimization_time': optimization_time,
            'n_iterations': 30,
            'cv_folds': cv_strategy.n_splits,
            'model_name': model_name,
            'optimization_method': 'BayesSearchCV',
            'timestamp': datetime.now().isoformat()
        }
        
        # Save to file
        results_file = os.path.join(results_dir, f"{model_name}_bayesian_optimization.json")
        with open(results_file, 'w') as f:
            json.dump(optimization_results, f, indent=2)
        
        print(f"✓ Results saved to: {results_file}")
        
        return bayesian_search.best_estimator_, optimization_results
    
    except Exception as e:
        print(f"❌ Bayesian optimization failed: {e}")
        print("Falling back to original pipeline")
        
        return pipeline, {'error': str(e)}

# Execute Bayesian optimization
if model_instance is not None and search_space:
    print(f"\n🔧 Starting Advanced Hyperparameter Optimization")
    
    # Create pipeline for optimization
    optimization_pipeline = create_advanced_pipeline(
        model_instance=model_instance,
        preprocessor=selected_preprocessor,
        stkde_config=stkde_config,
        use_smote=uses_smote
    )
    
    # Perform optimization
    optimized_pipeline, optimization_results = perform_bayesian_optimization(
        pipeline=optimization_pipeline,
        search_space=search_space,
        X_train=X_train,
        y_train=y_train,
        model_name=best_model_name,
        results_dir=final_results_dir
    )
    
    if 'error' not in optimization_results:
        print(f"\n🎯 Optimization Success!")
        print(f"   Improvement: {optimization_results.get('best_score', 0):.4f} F1-score")
    else:
        print(f"\n⚠ Optimization encountered issues")
        optimized_pipeline = optimization_pipeline
else:
    print("⚠ Cannot perform Bayesian optimization - missing model or search space")
    optimized_pipeline = None
    optimization_results = {}

# Model Interpretability Analysis

This section analyzes model behavior through feature importance, SHAP values, and permutation importance to understand model decisions.

In [ ]:
def perform_shap_analysis(pipeline: Pipeline, X_sample: pd.DataFrame, y_sample: pd.Series,
                         model_name: str, results_dir: str, max_samples: int = 500) -> Dict[str, Any]:
    """
    Perform SHAP analysis on the optimized model.
    
    Args:
        pipeline: Trained model pipeline (TargetEngineeringPipeline instance)
        X_sample, y_sample: Sample data for analysis
        model_name: Name of the model
        results_dir: Directory for saving results
        max_samples: Maximum samples for SHAP analysis
        
    Returns:
        Dictionary containing SHAP analysis results
    """
    print(f"\n=== SHAP Analysis for {model_name} ===")
    
    if not SHAP_AVAILABLE:
        print("⚠ SHAP not available, skipping analysis")
        return {'shap_available': False}
    
    try:
        # Sample data for efficiency
        if len(X_sample) > max_samples:
            sample_indices = np.random.choice(len(X_sample), max_samples, replace=False)
            X_shap = X_sample.iloc[sample_indices].copy()
            y_shap = y_sample.iloc[sample_indices].copy()
            print(f"Using {max_samples} samples for SHAP analysis")
        else:
            X_shap = X_sample.copy()
            y_shap = y_sample.copy()
            print(f"Using all {len(X_shap)} samples for SHAP analysis")
        
        # Transform data through target engineering and feature preprocessing (excluding final classifier)
        print("Transforming data for SHAP analysis...")
        # Step 1: Apply target engineering (STKDE)
        # 'pipeline' is the TargetEngineeringPipeline instance
        X_target_eng_features, _ = pipeline.target_engineer_.transform(X_shap)
        
        # Step 2: Apply feature pipeline (preprocessing, SMOTE if used) excluding the classifier
        # Its 'feature_pipeline' attribute is the one containing the classifier as the last step.
        if hasattr(pipeline.feature_pipeline_, 'steps'): # Usa l'attributo fittato feature_pipeline_
            # Assicurati che X_target_eng_features sia passato qui
            X_transformed = pipeline.feature_pipeline_[:-1].transform(X_target_eng_features) 
            # Get the final classifier itself
            classifier = pipeline.feature_pipeline_[-1]
        else: # Should not happen based on create_advanced_pipeline
            print("⚠ Could not extract classifier or preceding transformations for SHAP.")
            return {'shap_available': SHAP_AVAILABLE, 'error': 'Feature pipeline structure not as expected.'}
        
        # Create SHAP explainer based on model type
        print("Creating SHAP explainer...")
        
        if hasattr(classifier, 'predict_proba'):
            # For tree-based models, use TreeExplainer if available
            if hasattr(classifier, 'estimators_') or 'RandomForest' in str(type(classifier)):
                try:
                    explainer = shap.TreeExplainer(classifier)
                    shap_values = explainer.shap_values(X_transformed)
                    # For binary classification, take values for positive class
                    if isinstance(shap_values, list) and len(shap_values) == 2:
                        shap_values = shap_values[1]
                    explainer_type = "TreeExplainer"
                except:
                    # Fallback to KernelExplainer
                    explainer = shap.KernelExplainer(classifier.predict_proba, X_transformed[:100]) # Use a subset for KernelExplainer background
                    shap_values = explainer.shap_values(X_transformed[:200]) # And for explanation if large
                    if isinstance(shap_values, list) and len(shap_values) == 2:
                        shap_values = shap_values[1]
                    explainer_type = "KernelExplainer"
            else:
                # Use KernelExplainer for other models
                explainer = shap.KernelExplainer(classifier.predict_proba, X_transformed[:100]) # Use a subset for KernelExplainer background
                shap_values = explainer.shap_values(X_transformed[:200]) # And for explanation if large
                if isinstance(shap_values, list) and len(shap_values) == 2:
                    shap_values = shap_values[1]
                explainer_type = "KernelExplainer"
        else:
            print("⚠ Model doesn\'t support predict_proba, using LinearExplainer")
            explainer = shap.LinearExplainer(classifier, X_transformed)
            shap_values = explainer.shap_values(X_transformed)
            explainer_type = "LinearExplainer"
        
        print(f"✓ SHAP analysis completed using {explainer_type}")
        
        # Calculate feature importance from SHAP values
        # Ensure shap_values is a 2D array for mean calculation if it became a list of arrays
        if isinstance(shap_values, list):
            # This case might occur if KernelExplainer returns a list of arrays for multi-output
            # Assuming binary classification, we are interested in the positive class (often index 1)
            # This was handled above, but as a safeguard:
            if len(shap_values) == 2: shap_values = shap_values[1]
            else: # Fallback or error if unexpected structure
                 print("⚠ SHAP values have an unexpected list structure for binary classification.")
                 # Attempt to use the first element if it's a 2D array, otherwise error out or use zeros
                 if isinstance(shap_values[0], np.ndarray) and shap_values[0].ndim == 2:
                     shap_values = shap_values[0]
                 else:
                     shap_values = np.zeros_like(X_transformed, dtype=float) # Default to zeros to avoid crash
        
        feature_importance = np.abs(shap_values).mean(0)
        
        # Get feature names (may be different after transformation)
        if hasattr(X_transformed, 'columns'):
            feature_names_transformed = X_transformed.columns.tolist()
        elif hasattr(pipeline.feature_pipeline[:-1], 'get_feature_names_out'):
            try:
                feature_names_transformed = pipeline.feature_pipeline[:-1].get_feature_names_out()
            except:
                 feature_names_transformed = [f"feature_{i}" for i in range(X_transformed.shape[1])]
        else:
            feature_names_transformed = [f"feature_{i}" for i in range(X_transformed.shape[1])]
        
        # Create feature importance DataFrame
        importance_df = pd.DataFrame({
            'feature': feature_names_transformed,
            'importance': feature_importance
        }).sort_values('importance', ascending=False)
        
        print(f"Top 10 most important features:")
        for i, (_, row) in enumerate(importance_df.head(10).iterrows()):
            print(f"  {i+1:2d}. {row['feature']}: {row['importance']:.4f}")
        
        # Create visualizations
        plt.figure(figsize=(12, 8))
        
        # Feature importance plot
        plt.subplot(2, 2, 1)
        top_features = importance_df.head(15)
        plt.barh(range(len(top_features)), top_features['importance'])
        plt.yticks(range(len(top_features)), top_features['feature'])
        plt.xlabel('Mean |SHAP Value|')
        plt.title('Feature Importance (SHAP)')
        plt.gca().invert_yaxis()
        
        # SHAP summary plot
        plt.subplot(2, 2, 2)
        try:
            # Select top features for summary plot
            top_indices = importance_df.head(15).index
            # Ensure X_transformed is a DataFrame for summary_plot if possible
            if not isinstance(X_transformed, pd.DataFrame) and hasattr(X_transformed, 'shape'):
                X_transformed_df_for_plot = pd.DataFrame(X_transformed, columns=feature_names_transformed)
            else:
                X_transformed_df_for_plot = X_transformed

            shap.summary_plot(shap_values[:, top_indices] if shap_values.ndim > 1 else shap_values, 
                            X_transformed_df_for_plot.iloc[:, top_indices] if hasattr(X_transformed_df_for_plot, 'iloc') else X_transformed_df_for_plot[:, top_indices],
                            feature_names=[feature_names_transformed[i] for i in top_indices],
                            show=False, max_display=15)
            plt.title('SHAP Summary Plot')
        except Exception as e:
            plt.text(0.5, 0.5, f'Summary plot error:\n{str(e)[:100]}...', 
                    ha='center', va='center', transform=plt.gca().transAxes)
            plt.title('SHAP Summary Plot (Error)')
        
        # SHAP waterfall plot for a sample prediction
        plt.subplot(2, 2, 3)
        try:
            sample_idx = 0
            # Ensure explainer.expected_value is appropriate (scalar or array for multi-output)
            expected_value_for_waterfall = explainer.expected_value
            if isinstance(explainer.expected_value, list) or isinstance(explainer.expected_value, np.ndarray):
                 if len(explainer.expected_value) == 2: # Binary classification from KernelExplainer
                     expected_value_for_waterfall = explainer.expected_value[1]
                 elif len(explainer.expected_value) == 1:
                     expected_value_for_waterfall = explainer.expected_value[0]
            
            # Ensure shap_values[sample_idx] is 1D
            shap_values_sample = shap_values[sample_idx]
            if shap_values_sample.ndim > 1 and shap_values_sample.shape[0] == 1:
                shap_values_sample = shap_values_sample[0]
            elif shap_values_sample.ndim > 1: # Should not happen for a single sample's SHAP values for one class
                print(f"⚠ Unexpected SHAP values shape for waterfall: {shap_values_sample.shape}")
                shap_values_sample = shap_values_sample[:,0] # Fallback: use first output if multi-output

            shap.waterfall_plot(shap.Explanation(values=shap_values_sample, 
                                               base_values=expected_value_for_waterfall,
                                               data=X_transformed[sample_idx] if not isinstance(X_transformed, pd.DataFrame) else X_transformed.iloc[sample_idx].values,
                                               feature_names=feature_names_transformed), 
                              show=False, max_display=10)
            plt.title(f'SHAP Waterfall (Sample {sample_idx})')
        except Exception as e:
            plt.text(0.5, 0.5, f'Waterfall plot error:\n{str(e)[:100]}...', 
                    ha='center', va='center', transform=plt.gca().transAxes)
            plt.title('SHAP Waterfall (Error)')
        
        # SHAP dependence plot for top feature
        plt.subplot(2, 2, 4)
        try:
            top_feature_idx = importance_df.index[0] # This is an index into importance_df, not directly into X_transformed's columns
            top_feature_name = importance_df.iloc[0]['feature']
            # Find the actual column index in X_transformed
            if top_feature_name in feature_names_transformed:
                actual_col_idx = feature_names_transformed.index(top_feature_name)
            else: # Fallback if name mismatch, use first column
                actual_col_idx = 0 
            
            shap.dependence_plot(actual_col_idx, shap_values, 
                               X_transformed if not isinstance(X_transformed, pd.DataFrame) else X_transformed,
                               feature_names=feature_names_transformed, show=False)
            plt.title(f'SHAP Dependence: {feature_names_transformed[actual_col_idx]}')
        except Exception as e:
            plt.text(0.5, 0.5, f'Dependence plot error:\n{str(e)[:100]}...', 
                    ha='center', va='center', transform=plt.gca().transAxes)
            plt.title('SHAP Dependence (Error)')
        
        plt.tight_layout()
        
        # Save plot
        plot_file = os.path.join(results_dir, f"{model_name}_shap_analysis.png")
        plt.savefig(plot_file, dpi=300, bbox_inches='tight')
        plt.show()
        
        # Save importance data
        importance_file = os.path.join(results_dir, f"{model_name}_feature_importance.csv")
        importance_df.to_csv(importance_file, index=False)
        
        print(f"✓ SHAP plots saved to: {plot_file}")
        print(f"✓ Feature importance saved to: {importance_file}")
        
        return {
            'shap_available': True,
            'explainer_type': explainer_type,
            'feature_importance': importance_df.to_dict('records'),
            'n_samples_analyzed': len(X_shap),
            'plot_file': plot_file,
            'importance_file': importance_file
        }
        
    except Exception as e:
        print(f"❌ SHAP analysis failed: {e}")
        import traceback
        traceback.print_exc()
        return {'shap_available': True, 'error': str(e)}

def perform_permutation_importance_analysis(pipeline: Pipeline, X_test: pd.DataFrame, y_test: pd.Series,
                                          model_name: str, results_dir: str) -> Dict[str, Any]:
    """
    Perform permutation importance analysis.
    
    Args:
        pipeline: Trained model pipeline
        X_test, y_test: Test data
        model_name: Name of the model
        results_dir: Directory for saving results
        
    Returns:
        Dictionary containing permutation importance results
    """
    print(f"\n=== Permutation Importance Analysis for {model_name} ===")
    
    if not PERMUTATION_IMPORTANCE_AVAILABLE:
        print("⚠ Permutation importance not available")
        return {'permutation_available': False}
    
    try:
        print("Computing permutation importance...")
        start_time = time.time()
        
        # Compute permutation importance
        perm_importance = permutation_importance(
            pipeline, X_test, y_test, 
            n_repeats=10, 
            random_state=42, 
            scoring='f1',
            n_jobs=-1
        )
        
        computation_time = time.time() - start_time
        
        # Create results DataFrame
        perm_df = pd.DataFrame({
            'feature': X_test.columns if hasattr(X_test, 'columns') else [f'feature_{i}' for i in range(X_test.shape[1])],
            'importance_mean': perm_importance.importances_mean,
            'importance_std': perm_importance.importances_std
        }).sort_values('importance_mean', ascending=False)
        
        print(f"✓ Permutation importance computed in {computation_time:.1f} seconds")
        print(f"Top 10 features by permutation importance:")
        for i, (_, row) in enumerate(perm_df.head(10).iterrows()):
            print(f"  {i+1:2d}. {row['feature']}: {row['importance_mean']:.4f} (±{row['importance_std']:.4f})")
        
        # Create visualization
        plt.figure(figsize=(10, 8))
        
        top_features = perm_df.head(20)
        y_pos = np.arange(len(top_features))
        
        plt.barh(y_pos, top_features['importance_mean'], 
                xerr=top_features['importance_std'], capsize=3)
        plt.yticks(y_pos, top_features['feature'])
        plt.xlabel('Permutation Importance (F1 Score Decrease)')
        plt.title(f'Permutation Feature Importance - {model_name}')
        plt.gca().invert_yaxis()
        plt.grid(axis='x', alpha=0.3)
        
        # Save plot
        perm_plot_file = os.path.join(results_dir, f"{model_name}_permutation_importance.png")
        plt.savefig(perm_plot_file, dpi=300, bbox_inches='tight')
        plt.show()
        
        # Save results
        perm_file = os.path.join(results_dir, f"{model_name}_permutation_importance.csv")
        perm_df.to_csv(perm_file, index=False)
        
        print(f"✓ Permutation importance plot saved to: {perm_plot_file}")
        print(f"✓ Permutation importance data saved to: {perm_file}")
        
        return {
            'permutation_available': True,
            'computation_time': computation_time,
            'feature_importance': perm_df.to_dict('records'),
            'plot_file': perm_plot_file,
            'data_file': perm_file
        }
        
    except Exception as e:
        print(f"❌ Permutation importance analysis failed: {e}")
        return {'permutation_available': True, 'error': str(e)}

# Perform interpretability analysis
if optimized_pipeline is not None and best_model_name:
    print(f"\n🔍 Starting Model Interpretability Analysis")
    
    # Perform SHAP analysis
    shap_results = perform_shap_analysis(
        pipeline=optimized_pipeline,
        X_sample=X_train,
        y_sample=y_train,
        model_name=best_model_name,
        results_dir=final_results_dir,
        max_samples=500
    )
    
    # Perform permutation importance analysis
    perm_results = perform_permutation_importance_analysis(
        pipeline=optimized_pipeline,
        X_test=X_test,
        y_test=y_test,
        model_name=best_model_name,
        results_dir=final_results_dir
    )
    
    print(f"\n✅ Model interpretability analysis completed")
else:
    print("⚠ Cannot perform interpretability analysis - missing optimized pipeline")
    shap_results = {}
    perm_results = {}

# Threshold Optimization

## Introduction

This section optimizes the classification threshold for the best performing model to maximize F1 score.

In [ ]:
def optimize_classification_threshold(pipeline: Pipeline, X_val: pd.DataFrame, y_val: pd.Series,
                                    model_name: str, results_dir: str) -> Tuple[float, Dict[str, Any]]:
    """
    Optimize the classification threshold to maximize F1 score.
    
    Args:
        pipeline: Trained model pipeline
        X_val, y_val: Validation data
        model_name: Name of the model
        results_dir: Directory for saving results
        
    Returns:
        Tuple of (optimal_threshold, optimization_results)
    """
    print(f"\n=== Threshold Optimization for {model_name} ===")
    
    try:
        # Predict probabilities on validation set
        y_probs = pipeline.predict_proba(X_val)[:, 1]
        
        # Define thresholds to test
        thresholds = np.arange(0.0, 1.01, 0.01)
        
        # Calculate F1 scores for each threshold
        f1_scores = []
        for threshold in thresholds:
            y_pred = (y_probs >= threshold).astype(int)
            f1 = f1_score(y_val, y_pred)
            f1_scores.append(f1)
        
        # Find the threshold with the maximum F1 score
        max_f1_index = np.argmax(f1_scores)
        optimal_threshold = thresholds[max_f1_index]
        max_f1_score = f1_scores[max_f1_index]
        
        print(f"✓ Optimal threshold: {optimal_threshold:.2f} (F1: {max_f1_score:.4f})")
        
        # Save results
        results_file = os.path.join(results_dir, f"{model_name}_threshold_optimization.json")
        with open(results_file, 'w') as f:
            json.dump({
                'optimal_threshold': float(optimal_threshold),
                'max_f1_score': float(max_f1_score),
                'thresholds': thresholds.tolist(),
                'f1_scores': f1_scores,
                'model_name': model_name,
                'timestamp': datetime.now().isoformat()
            }, f, indent=2)
        
        print(f"✓ Results saved to: {results_file}")
        
        # Plot F1 scores by threshold
        plt.figure(figsize=(10, 6))
        plt.plot(thresholds, f1_scores, marker='o')
        plt.xlabel('Classification Threshold')
        plt.ylabel('F1 Score')
        plt.title(f'F1 Score Optimization - {model_name}')
        plt.grid()
        plt.xlim(0, 1)
        plt.ylim(0, 1)
        
        # Highlight optimal threshold
        plt.axvline(optimal_threshold, color='r', linestyle='--', label=f'Optimal Threshold: {optimal_threshold:.2f}')
        plt.legend()
        
        # Save plot
        plot_file = os.path.join(results_dir, f"{model_name}_f1_threshold_optimization.png")
        plt.savefig(plot_file, dpi=300, bbox_inches='tight')
        plt.show()
        
        return optimal_threshold, {
            'optimal_threshold': float(optimal_threshold),
            'max_f1_score': float(max_f1_score),
            'thresholds': thresholds.tolist(),
            'f1_scores': f1_scores
        }
    
    except Exception as e:
        print(f"❌ Threshold optimization failed: {e}")
        return 0.5, {'error': str(e)}

# Optimize threshold for the best model
if optimized_pipeline is not None and best_model_name:
    print(f"\n🎯 Starting Threshold Optimization for {best_model_name}")
    
    optimal_threshold, threshold_results = optimize_classification_threshold(
        pipeline=optimized_pipeline,
        X_val=X_test,  # Using test set for final validation
        y_val=y_test,
        model_name=best_model_name,
        results_dir=final_results_dir
    )
else:
    print("⚠ Cannot perform threshold optimization - missing optimized pipeline")
    optimal_threshold = 0.5
    threshold_results = {}

# Production Model Training

## Introduction

Train the final production model using the optimized pipeline and parameters.

In [ ]:
def train_final_production_model(pipeline: Pipeline, X_train: pd.DataFrame, y_train: pd.Series,
                                X_val: pd.DataFrame, y_val: pd.Series,
                                model_name: str, results_dir: str, optimal_threshold: float) -> Dict[str, Any]:
    """
    Train the final production model with optimized parameters and threshold.
    
    Args:
        pipeline: Optimized model pipeline
        X_train, y_train: Training data
        X_val, y_val: Validation data
        model_name: Name of the model
        results_dir: Directory for saving results
        optimal_threshold: Optimal classification threshold
        
    Returns:
        Dictionary containing training results
    """
    print(f"\n=== Final Model Training for {model_name} ===")
    
    try:
        # Train the model
        pipeline.fit(X_train, y_train)
        
        # Predict on validation set
        y_val_pred = pipeline.predict(X_val)
        y_val_probs = pipeline.predict_proba(X_val)[:, 1]
        
        # Calculate metrics
        accuracy = accuracy_score(y_val, y_val_pred)
        precision = precision_score(y_val, y_val_pred)
        recall = recall_score(y_val, y_val_pred)
        f1 = f1_score(y_val, y_val_pred)
        roc_auc = roc_auc_score(y_val, y_val_probs)
        
        print(f"✓ Model trained successfully")
        print(f"  Accuracy: {accuracy:.4f}")
        print(f"  Precision: {precision:.4f}")
        print(f"  Recall: {recall:.4f}")
        print(f"  F1 Score: {f1:.4f}")
        print(f"  ROC AUC: {roc_auc:.4f}")
        
        # Save final model
        model_file = os.path.join(results_dir, f"{model_name}_final_model.joblib")
        joblib.dump(pipeline, model_file)
        print(f"✓ Model saved to: {model_file}")
        
        # Save final predictions
        pred_file = os.path.join(results_dir, f"{model_name}_final_predictions.csv")
        pd.DataFrame({
            'true': y_val,
            'predicted': y_val_pred,
            'probability': y_val_probs
        }).to_csv(pred_file, index=False)
        print(f"✓ Predictions saved to: {pred_file}")
        
        # Save training results
        results_file = os.path.join(results_dir, f"{model_name}_training_results.json")
        with open(results_file, 'w') as f:
            json.dump({
                'accuracy': float(accuracy),
                'precision': float(precision),
                'recall': float(recall),
                'f1_score': float(f1),
                'roc_auc': float(roc_auc),
                'model_file': model_file,
                'pred_file': pred_file,
                'timestamp': datetime.now().isoformat()
            }, f, indent=2)
        
        print(f"✓ Training results saved to: {results_file}")
        
        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'roc_auc': roc_auc,
            'model_file': model_file,
            'pred_file': pred_file
        }
    
    except Exception as e:
        print(f"❌ Final model training failed: {e}")
        return {'error': str(e)}

# Train final production model
if optimized_pipeline is not None and best_model_name:
    print(f"\n🚀 Starting Final Model Training for {best_model_name}")
    
    final_training_results = train_final_production_model(
        pipeline=optimized_pipeline,
        X_train=X_train,
        y_train=y_train,
        X_val=X_test,  # Using test set for final validation
        y_val=y_test,
        model_name=best_model_name,
        results_dir=final_results_dir,
        optimal_threshold=optimal_threshold
    )
else:
    print("⚠ Cannot train final model - missing optimized pipeline")
    final_training_results = {}

# Comprehensive Model Validation

## Introduction

Validate the final model performance using various metrics and compare with baseline models.

In [ ]:
def validate_final_model(model_name: str, X_val: pd.DataFrame, y_val: pd.Series,
                        results_dir: str, baseline_results: Optional[Dict[str, Any]] = None) -> None:
    """
    Validate the final model performance and compare with baseline models.
    
    Args:
        model_name: Name of the model
        X_val, y_val: Validation data
        results_dir: Directory for saving results
        baseline_results: Optional baseline results for comparison
        
    Returns:
        None
    """
    print(f"\n=== Comprehensive Validation for {model_name} ===")
    
    try:
        # Load final model
        model_file = os.path.join(results_dir, f"{model_name}_final_model.joblib")
        if not os.path.exists(model_file):
            print(f"⚠ Model file not found: {model_file}")
            return
        
        pipeline = joblib.load(model_file)
        
        # Predict on validation set
        y_val_pred = pipeline.predict(X_val)
        y_val_probs = pipeline.predict_proba(X_val)[:, 1]
        
        # Calculate metrics
        accuracy = accuracy_score(y_val, y_val_pred)
        precision = precision_score(y_val, y_val_pred)
        recall = recall_score(y_val, y_val_pred)
        f1 = f1_score(y_val, y_val_pred)
        roc_auc = roc_auc_score(y_val, y_val_probs)
        
        print(f"✓ Model loaded and evaluated successfully")
        print(f"  Accuracy: {accuracy:.4f}")
        print(f"  Precision: {precision:.4f}")
        print(f"  Recall: {recall:.4f}")
        print(f"  F1 Score: {f1:.4f}")
        print(f"  ROC AUC: {roc_auc:.4f}")
        
        # Save validation results
        validation_results = {
            'accuracy': float(accuracy),
            'precision': float(precision),
            'recall': float(recall),
            'f1_score': float(f1),
            'roc_auc': float(roc_auc),
            'model_name': model_name,
            'timestamp': datetime.now().isoformat()
        }
        
        results_file = os.path.join(results_dir, f"{model_name}_validation_results.json")
        with open(results_file, 'w') as f:
            json.dump(validation_results, f, indent=2)
        
        print(f"✓ Validation results saved to: {results_file}")
        
        # Compare with baseline models if available
        if baseline_results is not None:
            print(f"\n📊 Comparing with baseline models...")
            
            for baseline_name, baseline_metrics in baseline_results.items():
                print(f"  {baseline_name}: F1={baseline_metrics.get('f1_score', 0):.4f}, ROC AUC={baseline_metrics.get('roc_auc', 0):.4f}")
        
    except Exception as e:
        print(f"❌ Validation failed: {e}")

# Validate final model
if best_model_name and final_training_results.get('model_file'):
    print(f"\n✅ Validating Final Model: {best_model_name}")
    
    # Load baseline results for comparison
    baseline_results_file = os.path.join(final_results_dir, "baseline_model_results.json")
    baseline_results = None
    if os.path.exists(baseline_results_file):
        with open(baseline_results_file, 'r') as f:
            baseline_results = json.load(f)
            print(f"✓ Loaded baseline results from: {baseline_results_file}")
    else:
        print("⚠ Baseline results file not found")
    
    validate_final_model(
        model_name=best_model_name,
        X_val=X_test,  # Using test set for final validation
        y_val=y_test,
        results_dir=final_results_dir,
        baseline_results=baseline_results
    )
else:
    print("⚠ Cannot validate final model - missing model file")

# Report Generation and Model Documentation

## Introduction

Generate comprehensive reports and documentation for the final model, including performance metrics, visualizations, and model details.

In [ ]:
# Define report content
report_content = f"""
# DMML Binary Risk Classification - Final Model Report

## Model Overview

- **Model Name**: {best_model_name}
- **Training Data**: {X_train.shape[0]} samples
- **Validation Data**: {X_val.shape[0]} samples
- **Features**: {len(feature_names)} (after preprocessing)
- **Target**: Binary risk classification

## Performance Metrics

| Metric       | Value       |
|--------------|-------------|
| Accuracy      | {final_training_results['accuracy']:.4f}      |
| Precision     | {final_training_results['precision']:.4f}     |
| Recall        | {final_training_results['recall']:.4f}        |
| F1 Score      | {final_training_results['f1_score']:.4f}      |
| ROC AUC       | {final_training_results['roc_auc']:.4f}       |

## Threshold Optimization

- **Optimal Threshold**: {optimal_threshold:.2f}
- **Max F1 Score**: {threshold_results.get('max_f1_score', 0):.4f}

## Model Artifacts

- **Model File**: `{final_training_results['model_file']}`
- **Prediction File**: `{final_training_results['pred_file']}`

## Visualizations

### Feature Importance (SHAP)

![SHAP Feature Importance](./{best_model_name}_shap_analysis.png)

### Permutation Feature Importance

![Permutation Feature Importance](./{best_model_name}_permutation_importance.png)

### F1 Score Optimization

![F1 Score Optimization](./{best_model_name}_f1_threshold_optimization.png)

## Conclusion

The final model has been trained and validated successfully with optimal performance metrics. The model is ready for deployment in the production environment.
"""

# Save report to file
report_file = os.path.join(final_results_dir, "final_model_report.md")
with open(report_file, 'w') as f:
    f.write(report_content)

print(f"✓ Report saved to: {report_file}")

# Display report
from IPython.display import Markdown, display
display(Markdown(report_file))

## Learning Curves and Validation Curves

Analyze model performance across different training set sizes and key hyperparameters.

In [ ]:
def analyze_learning_curves(pipeline: Pipeline, X_train: pd.DataFrame, y_train: pd.Series,
                          model_name: str, results_dir: str) -> Dict[str, Any]:
    """
    Generate and analyze learning curves for the optimized model.
    
    Args:
        pipeline: Trained model pipeline
        X_train, y_train: Training data
        model_name: Name of the model
        results_dir: Directory for saving results
        
    Returns:
        Dictionary containing learning curve analysis results
    """
    print(f"\n=== Learning Curve Analysis for {model_name} ===")
    
    try:
        # Set up cross-validation strategy
        temporal_cols = ['YEAR', 'MONTH', 'DAY', 'HOUR']
        if all(col in X_train.columns for col in temporal_cols):
            cv_strategy = TimeSeriesSplit(n_splits=3)
            print("Using TimeSeriesSplit for learning curves")
        else:
            cv_strategy = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
            print("Using StratifiedKFold for learning curves")
        
        # Define training sizes
        train_sizes = np.linspace(0.1, 1.0, 10)
        
        print("Computing learning curves...")
        start_time = time.time()
        
        # Compute learning curves
        train_sizes_abs, train_scores, val_scores = learning_curve(
            estimator=pipeline,
            X=X_train,
            y=y_train,
            cv=cv_strategy,
            train_sizes=train_sizes,
            scoring='f1',
            n_jobs=1,  # Avoid nested parallelization
            random_state=42
        )
        
        computation_time = time.time() - start_time
        
        # Calculate statistics
        train_scores_mean = np.mean(train_scores, axis=1)
        train_scores_std = np.std(train_scores, axis=1)
        val_scores_mean = np.mean(val_scores, axis=1)
        val_scores_std = np.std(val_scores, axis=1)
        
        print(f"✓ Learning curves computed in {computation_time:.1f} seconds")
        
        # Create learning curve plot
        plt.figure(figsize=(12, 8))
        
        plt.subplot(2, 2, 1)
        plt.plot(train_sizes_abs, train_scores_mean, 'o-', color='blue', label='Training Score')
        plt.fill_between(train_sizes_abs, train_scores_mean - train_scores_std,
                        train_scores_mean + train_scores_std, alpha=0.1, color='blue')
        
        plt.plot(train_sizes_abs, val_scores_mean, 'o-', color='red', label='Validation Score')
        plt.fill_between(train_sizes_abs, val_scores_mean - val_scores_std,
                        val_scores_mean + val_scores_std, alpha=0.1, color='red')
        
        plt.xlabel('Training Set Size')
        plt.ylabel('F1 Score')
        plt.title(f'Learning Curves - {model_name}')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Score vs training size percentage
        plt.subplot(2, 2, 2)
        train_size_percentages = train_sizes_abs / len(X_train) * 100
        plt.plot(train_size_percentages, train_scores_mean, 'o-', color='blue', label='Training Score')
        plt.plot(train_size_percentages, val_scores_mean, 'o-', color='red', label='Validation Score')
        plt.xlabel('Training Set Size (%)')
        plt.ylabel('F1 Score')
        plt.title('Performance vs Training Data Percentage')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Learning efficiency analysis
        plt.subplot(2, 2, 3)
        learning_efficiency = np.diff(val_scores_mean) / np.diff(train_sizes_abs)
        plt.plot(train_sizes_abs[1:], learning_efficiency, 'o-', color='green')
        plt.xlabel('Training Set Size')
        plt.ylabel('Learning Efficiency (ΔF1/ΔSamples)')
        plt.title('Learning Efficiency')
        plt.grid(True, alpha=0.3)
        
        # Overfitting analysis
        plt.subplot(2, 2, 4)
        overfitting_gap = train_scores_mean - val_scores_mean
        plt.plot(train_sizes_abs, overfitting_gap, 'o-', color='orange')
        plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
        plt.xlabel('Training Set Size')
        plt.ylabel('Training - Validation Score Gap')
        plt.title('Overfitting Analysis')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        
        # Save plot
        learning_plot_file = os.path.join(results_dir, f"{model_name}_learning_curves.png")
        plt.savefig(learning_plot_file, dpi=300, bbox_inches='tight')
        plt.show()
        
        # Save learning curve data
        learning_data = pd.DataFrame({
            'train_size': train_sizes_abs,
            'train_size_pct': train_size_percentages,
            'train_score_mean': train_scores_mean,
            'train_score_std': train_scores_std,
            'val_score_mean': val_scores_mean,
            'val_score_std': val_scores_std,
            'overfitting_gap': overfitting_gap
        })
        
        learning_file = os.path.join(results_dir, f"{model_name}_learning_curve_data.csv")
        learning_data.to_csv(learning_file, index=False)
        
        # Analyze results
        final_train_score = train_scores_mean[-1]
        final_val_score = val_scores_mean[-1]
        final_gap = overfitting_gap[-1]
        
        # Performance recommendations
        recommendations = []
        if final_gap > 0.1:
            recommendations.append("High overfitting detected - consider regularization or more data")
        elif final_gap < 0.02:
            recommendations.append("Good bias-variance balance achieved")
        
        if val_scores_mean[-1] > val_scores_mean[-2]:
            recommendations.append("Model still improving - could benefit from more training data")
        
        print(f"Learning Curve Analysis Results:")
        print(f"  Final training score: {final_train_score:.4f}")
        print(f"  Final validation score: {final_val_score:.4f}")
        print(f"  Overfitting gap: {final_gap:.4f}")
        print(f"  Recommendations: {'; '.join(recommendations) if recommendations else 'Model performance looks good'}")
        
        print(f"✓ Learning curves saved to: {learning_plot_file}")
        print(f"✓ Learning data saved to: {learning_file}")
        
        return {
            'computation_time': computation_time,
            'final_train_score': float(final_train_score),
            'final_val_score': float(final_val_score),
            'overfitting_gap': float(final_gap),
            'recommendations': recommendations,
            'learning_data': learning_data.to_dict('records'),
            'plot_file': learning_plot_file,
            'data_file': learning_file
        }
        
    except Exception as e:
        print(f"❌ Learning curve analysis failed: {e}")
        return {'error': str(e)}

def analyze_validation_curves(pipeline: Pipeline, X_train: pd.DataFrame, y_train: pd.Series,
                            model_name: str, results_dir: str) -> Dict[str, Any]:
    """
    Generate validation curves for key hyperparameters.
    
    Args:
        pipeline: Model pipeline
        X_train, y_train: Training data
        model_name: Name of the model
        results_dir: Directory for saving results
        
    Returns:
        Dictionary containing validation curve results
    """
    print(f"\n=== Validation Curve Analysis for {model_name} ===")
    
    try:
        # Set up cross-validation
        temporal_cols = ['YEAR', 'MONTH', 'DAY', 'HOUR']
        if all(col in X_train.columns for col in temporal_cols):
            cv_strategy = TimeSeriesSplit(n_splits=3)
        else:
            cv_strategy = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        
        # Define validation parameters based on model type
        validation_params = {}
        
        if 'RandomForest' in model_name:
            validation_params = {
                'feature_pipeline__classifier__n_estimators': [10, 25, 50, 100, 200, 300],
                'feature_pipeline__classifier__max_depth': [3, 5, 10, 15, 20, None]
            }
        elif 'GradientBoosting' in model_name or 'XGB' in model_name or 'LightGBM' in model_name:
            validation_params = {
                'feature_pipeline__classifier__n_estimators': [10, 25, 50, 100, 200, 300],
                'feature_pipeline__classifier__learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3]
            }
        elif 'LogisticRegression' in model_name:
            validation_params = {
                'feature_pipeline__classifier__C': [0.01, 0.1, 1.0, 10.0, 100.0]
            }
        
        if not validation_params:
            print("⚠ No validation parameters defined for this model type")
            return {'validation_available': False}
        
        print(f"Analyzing {len(validation_params)} hyperparameters...")
        
        validation_results = {}
        
        fig, axes = plt.subplots(1, len(validation_params), figsize=(6*len(validation_params), 6))
        if len(validation_params) == 1:
            axes = [axes]
        
        for i, (param_name, param_range) in enumerate(validation_params.items()):
            print(f"  Computing validation curve for {param_name}...")
            
            try:
                train_scores, val_scores = validation_curve(
                    estimator=pipeline,
                    X=X_train,
                    y=y_train,
                    param_name=param_name,
                    param_range=param_range,
                    cv=cv_strategy,
                    scoring='f1',
                    n_jobs=1
                )
                
                train_mean = np.mean(train_scores, axis=1)
                train_std = np.std(train_scores, axis=1)
                val_mean = np.mean(val_scores, axis=1)
                val_std = np.std(val_scores, axis=1)
                
                # Plot validation curve
                axes[i].plot(param_range, train_mean, 'o-', color='blue', label='Training Score')
                axes[i].fill_between(param_range, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
                
                axes[i].plot(param_range, val_mean, 'o-', color='red', label='Validation Score')
                axes[i].fill_between(param_range, val_mean - val_std, val_mean + val_std, alpha=0.1, color='red')
                
                axes[i].set_xlabel(param_name.split('__')[-1])
                axes[i].set_ylabel('F1 Score')
                axes[i].set_title(f'Validation Curve: {param_name.split("__")[-1]}')
                axes[i].legend()
                axes[i].grid(True, alpha=0.3)
                
                # Set log scale if parameter values span orders of magnitude
                if param_name.endswith('C') or param_name.endswith('learning_rate'):
                    axes[i].set_xscale('log')
                
                # Store results
                validation_results[param_name] = {
                    'param_range': param_range,
                    'train_scores_mean': train_mean.tolist(),
                    'train_scores_std': train_std.tolist(),
                    'val_scores_mean': val_mean.tolist(),
                    'val_scores_std': val_std.tolist(),
                    'best_param': param_range[np.argmax(val_mean)],
                    'best_score': float(np.max(val_mean))
                }
                
                print(f"    Best {param_name.split('__')[-1]}: {validation_results[param_name]['best_param']} (F1: {validation_results[param_name]['best_score']:.4f})")
                
            except Exception as e:
                print(f"    ❌ Failed to compute validation curve for {param_name}: {e}")
                axes[i].text(0.5, 0.5, f'Error:\n{str(e)[:50]}...', ha='center', va='center', transform=axes[i].transAxes)
                axes[i].set_title(f'Validation Curve: {param_name.split("__")[-1]} (Error)')
        
        plt.tight_layout()
        
        # Save plot
        validation_plot_file = os.path.join(results_dir, f"{model_name}_validation_curves.png")
        plt.savefig(validation_plot_file, dpi=300, bbox_inches='tight')
        plt.show()
        
        # Save validation results
        validation_file = os.path.join(results_dir, f"{model_name}_validation_curves.json")
        with open(validation_file, 'w') as f:
            json.dump(validation_results, f, indent=2)
        
        print(f"✓ Validation curves saved to: {validation_plot_file}")
        print(f"✓ Validation data saved to: {validation_file}")
        
        return {
            'validation_available': True,
            'validation_results': validation_results,
            'plot_file': validation_plot_file,
            'data_file': validation_file
        }
        
    except Exception as e:
        print(f"❌ Validation curve analysis failed: {e}")
        return {'validation_available': True, 'error': str(e)}

# Perform learning curve analysis
if optimized_pipeline is not None and best_model_name:
    print(f"\n📈 Starting Learning Curve Analysis")
    
    learning_results = analyze_learning_curves(
        pipeline=optimized_pipeline,
        X_train=X_train,
        y_train=y_train,
        model_name=best_model_name,
        results_dir=final_results_dir
    )
    
    # Perform validation curve analysis
    validation_results = analyze_validation_curves(
        pipeline=optimized_pipeline,
        X_train=X_train,
        y_train=y_train,
        model_name=best_model_name,
        results_dir=final_results_dir
    )
    
    print(f"\n✅ Learning and validation curve analysis completed")
else:
    print("⚠ Cannot perform curve analysis - missing optimized pipeline")
    learning_results = {}
    validation_results = {}

## Final Model Training and Test Set Evaluation

Train the final production model and evaluate on the held-out test set.

In [ ]:
def train_final_production_model(pipeline: Pipeline, X_train: pd.DataFrame, y_train: pd.Series,
                                X_test: pd.DataFrame, y_test: pd.Series, threshold: float,
                                model_name: str, results_dir: str) -> Dict[str, Any]:
    """
    Train the final production model and evaluate on test set.
    
    Args:
        pipeline: Optimized model pipeline
        X_train, y_train: Training data
        X_test, y_test: Test data
        threshold: Optimized classification threshold
        model_name: Name of the model
        results_dir: Directory for saving results
        
    Returns:
        Dictionary containing final model results
    """
    print(f"\n=== Final Production Model Training and Evaluation ===")
    print(f"Model: {model_name}")
    print(f"Classification Threshold: {threshold:.3f}")
    
    try:
        # Train on full training set
        print("Training final model on complete training set...")
        start_time = time.time()
        
        pipeline.fit(X_train, y_train)
        
        training_time = time.time() - start_time
        print(f"✓ Training completed in {training_time:.1f} seconds")
        
        # Predictions on test set
        print("Evaluating on test set...")
        y_test_proba = pipeline.predict_proba(X_test)[:, 1]
        y_test_pred_default = pipeline.predict(X_test) # Uses default 0.5 threshold from model's predict
        y_test_pred_optimized = (y_test_proba >= threshold).astype(int)
        
        # Calculate comprehensive metrics
        def calculate_metrics(y_true, y_pred, y_proba):
            return {
                'accuracy': accuracy_score(y_true, y_pred),
                'precision': precision_score(y_true, y_pred, zero_division=0),
                'recall': recall_score(y_true, y_pred, zero_division=0),
                'f1': f1_score(y_true, y_pred, zero_division=0),
                'roc_auc': roc_auc_score(y_true, y_proba),
                'average_precision': average_precision_score(y_true, y_proba),
                'matthews_corrcoef': matthews_corrcoef(y_true, y_pred)
            }
        
        # Metrics with default threshold (0.5)
        metrics_default = calculate_metrics(y_test, y_test_pred_default, y_test_proba)
        
        # Metrics with optimized threshold
        metrics_optimized = calculate_metrics(y_test, y_test_pred_optimized, y_test_proba)
        
        print(f"\n📊 Test Set Performance:")
        print(f"Default Threshold (0.5):")
        for metric, value in metrics_default.items():
            print(f"  {metric:20}: {value:.4f}")
        
        print(f"\nOptimized Threshold ({threshold:.3f}):")
        for metric, value in metrics_optimized.items():
            print(f"  {metric:20}: {value:.4f}")
        
        # Generate classification report
        print(f"\n📋 Detailed Classification Report (Optimized Threshold):")
        class_report_text = classification_report(y_test, y_test_pred_optimized)
        print(class_report_text)
        class_report_dict = classification_report(y_test, y_test_pred_optimized, output_dict=True)
        
        # Create comprehensive evaluation plots
        fig = plt.figure(figsize=(20, 16))
        
        # 1. Confusion Matrix (Default)
        plt.subplot(3, 4, 1)
        cm_default = confusion_matrix(y_test, y_test_pred_default)
        ConfusionMatrixDisplay(cm_default, display_labels=['Low Risk', 'High Risk']).plot(ax=plt.gca())
        plt.title(f'Confusion Matrix (Threshold: 0.5)')
        
        # 2. Confusion Matrix (Optimized)
        plt.subplot(3, 4, 2)
        cm_optimized = confusion_matrix(y_test, y_test_pred_optimized)
        ConfusionMatrixDisplay(cm_optimized, display_labels=['Low Risk', 'High Risk']).plot(ax=plt.gca())
        plt.title(f'Confusion Matrix (Threshold: {threshold:.3f})')
        
        # 3. ROC Curve
        plt.subplot(3, 4, 3)
        fpr, tpr, _ = roc_curve(y_test, y_test_proba)
        plt.plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC: {metrics_optimized["roc_auc"]:.4f})')
        plt.plot([0, 1], [0, 1], 'k--', alpha=0.5)
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curve')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 4. Precision-Recall Curve
        plt.subplot(3, 4, 4)
        precision_curve_vals, recall_curve_vals, _ = precision_recall_curve(y_test, y_test_proba)
        plt.plot(recall_curve_vals, precision_curve_vals, linewidth=2, 
                label=f'PR Curve (AP: {metrics_optimized["average_precision"]:.4f})')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curve')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 5. Prediction Probability Distribution
        plt.subplot(3, 4, 5)
        plt.hist(y_test_proba[y_test == 0], bins=30, alpha=0.7, label='Low Risk', density=True)
        plt.hist(y_test_proba[y_test == 1], bins=30, alpha=0.7, label='High Risk', density=True)
        plt.axvline(threshold, color='red', linestyle='--', label=f'Threshold: {threshold:.3f}')
        plt.xlabel('Prediction Probability')
        plt.ylabel('Density')
        plt.title('Prediction Probability Distribution')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 6. Feature Importance (if available)
        plt.subplot(3, 4, 6)
        try:
            # Access feature importances from the classifier step of the feature_pipeline
            final_classifier_step = pipeline.feature_pipeline[-1]
            if hasattr(final_classifier_step, 'feature_importances_'):
                importances = final_classifier_step.feature_importances_
                # Get feature names from the step preceding the classifier in feature_pipeline
                preprocessor_step = pipeline.feature_pipeline[:-1]
                if hasattr(preprocessor_step, 'get_feature_names_out'):
                    feature_names_final = preprocessor_step.get_feature_names_out()
                else: # Fallback if get_feature_names_out is not available or X was numpy array
                    feature_names_final = [f'feature_{i}' for i in range(len(importances))]
                
                # Top 15 features
                indices = np.argsort(importances)[::-1][:15]
                # Ensure we don't try to plot more features than available
                num_features_to_plot = min(15, len(importances))
                plt.barh(range(num_features_to_plot), importances[indices][:num_features_to_plot])
                plt.yticks(range(num_features_to_plot), [feature_names_final[i] for i in indices][:num_features_to_plot])
                plt.xlabel('Feature Importance')
                plt.title('Top Feature Importances')
                plt.gca().invert_yaxis()
            else:
                plt.text(0.5, 0.5, 'Feature importance\nnot available for this classifier', 
                        ha='center', va='center', transform=plt.gca().transAxes)
                plt.title('Feature Importance (N/A)')
        except Exception as e:
            plt.text(0.5, 0.5, f'Feature importance\nerror: {str(e)[:30]}...', 
                    ha='center', va='center', transform=plt.gca().transAxes)
            plt.title('Feature Importance (Error)')
        
        # 7. Metrics Comparison
        plt.subplot(3, 4, 7)
        metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1', 'MCC']
        default_values = [metrics_default['accuracy'], metrics_default['precision'], 
                         metrics_default['recall'], metrics_default['f1'], 
                         metrics_default['matthews_corrcoef']]
        optimized_values = [metrics_optimized['accuracy'], metrics_optimized['precision'], 
                           metrics_optimized['recall'], metrics_optimized['f1'], 
                           metrics_optimized['matthews_corrcoef']]
        
        x_bar = np.arange(len(metrics_names))
        width = 0.35
        
        plt.bar(x_bar - width/2, default_values, width, label='Default (0.5)', alpha=0.7)
        plt.bar(x_bar + width/2, optimized_values, width, label=f'Optimized ({threshold:.3f})', alpha=0.7)
        plt.xlabel('Metrics')
        plt.ylabel('Score')
        plt.title('Threshold Comparison')
        plt.xticks(x_bar, metrics_names)
        plt.legend()
        plt.grid(axis='y', alpha=0.3)
        
        # 8. Residual Analysis (for probabilities)
        plt.subplot(3, 4, 8)
        residuals = y_test - y_test_proba
        plt.scatter(y_test_proba, residuals, alpha=0.6)
        plt.axhline(y=0, color='red', linestyle='--')
        plt.xlabel('Predicted Probability')
        plt.ylabel('Residuals (Actual - Predicted)')
        plt.title('Residual Analysis')
        plt.grid(True, alpha=0.3)
        
        # 9-12. Performance by class and other analyses
        plt.subplot(3, 4, 9)
        class_performance_data = []
        for label, metrics in class_report_dict.items():
            if label in ['0', '1']:
                class_performance_data.append({
                    'Class': f"{'Low Risk' if label == '0' else 'High Risk'} ({label})",
                    'Precision': metrics['precision'],
                    'Recall': metrics['recall'],
                    'F1-Score': metrics['f1-score']
                })
        class_performance_df = pd.DataFrame(class_performance_data)
        
        x_bar_class = np.arange(len(class_performance_df))
        width_class = 0.25
        
        plt.bar(x_bar_class - width_class, class_performance_df['Precision'], width_class, label='Precision', alpha=0.7)
        plt.bar(x_bar_class, class_performance_df['Recall'], width_class, label='Recall', alpha=0.7)
        plt.bar(x_bar_class + width_class, class_performance_df['F1-Score'], width_class, label='F1-Score', alpha=0.7)
        
        plt.xlabel('Class')
        plt.ylabel('Score')
        plt.title('Per-Class Performance (Optimized)')
        plt.xticks(x_bar_class, class_performance_df['Class'])
        plt.legend()
        plt.grid(axis='y', alpha=0.3)
        
        # 10. Error Analysis
        plt.subplot(3, 4, 10)
        errors = {
            'True Positives': int(cm_optimized[1, 1]),
            'False Positives': int(cm_optimized[0, 1]),
            'False Negatives': int(cm_optimized[1, 0]),
            'True Negatives': int(cm_optimized[0, 0])
        }
        
        plt.pie(list(errors.values()), labels=list(errors.keys()), autopct='%1.1f%%', startangle=90)
        plt.title('Prediction Breakdown (Optimized)')
        
        # 11. Calibration Plot
        plt.subplot(3, 4, 11)
        from sklearn.calibration import calibration_curve
        fraction_of_positives, mean_predicted_value = calibration_curve(y_test, y_test_proba, n_bins=10, strategy='uniform')
        plt.plot(mean_predicted_value, fraction_of_positives, "s-", label=f"{model_name}")
        plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
        plt.xlabel('Mean Predicted Probability')
        plt.ylabel('Fraction of Positives')
        plt.title('Calibration Plot')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 12. Summary Statistics
        plt.subplot(3, 4, 12)
        summary_stats = {
            'Total Test Samples': len(y_test),
            'Positive Samples': int(y_test.sum()),
            'Negative Samples': int(len(y_test) - y_test.sum()),
            'Positive Rate': f"{y_test.mean():.3f}",
            'Training Time': f"{training_time:.1f}s"
        }
        
        plt.axis('off')
        text_str = '\n'.join([f'{k}: {v}' for k, v in summary_stats.items()])
        plt.text(0.05, 0.5, text_str, transform=plt.gca().transAxes, fontsize=10,
                verticalalignment='center', bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.7))
        plt.title('Summary Statistics (Test Set)')
        
        plt.tight_layout(pad=2.0)
        
        # Save comprehensive evaluation plot
        final_eval_plot = os.path.join(results_dir, f"{model_name}_final_evaluation.png")
        plt.savefig(final_eval_plot, dpi=300, bbox_inches='tight')
        plt.show()
        
        # Save final model
        final_model_file = os.path.join(results_dir, f"{model_name}_final_model.joblib")
        joblib.dump(pipeline, final_model_file)
        
        # Save final results
        final_results_data = {
            'model_name': model_name,
            'training_time_seconds': training_time,
            'test_samples_count': len(y_test),
            'positive_samples_test_count': int(y_test.sum()),
            'default_threshold_metrics': {
                'threshold': 0.5,
                'metrics': metrics_default
            },
            'optimized_threshold_metrics': {
                'threshold': float(threshold),
                'metrics': metrics_optimized
            },
            'classification_report_optimized': class_report_dict,
            'confusion_matrix_default_tn_fp_fn_tp': cm_default.tolist(),
            'confusion_matrix_optimized_tn_fp_fn_tp': cm_optimized.tolist(),
            'summary_statistics_test_set': summary_stats,
            'plot_file_final_evaluation': final_eval_plot,
            'model_file_final': final_model_file,
            'timestamp': datetime.now().isoformat()
        }
        
        final_results_file = os.path.join(results_dir, f"{model_name}_final_results.json")
        with open(final_results_file, 'w') as f:
            json.dump(final_results_data, f, indent=2, default=str) # Use default=str for non-serializable like numpy floats
        
        print(f"\n✅ Final Model Training and Evaluation Complete!")
        print(f"✓ Model saved to: {final_model_file}")
        print(f"✓ Evaluation plot saved to: {final_eval_plot}")
        print(f"✓ Results saved to: {final_results_file}")
        
        return final_results_data
        
    except Exception as e:
        print(f"❌ Final model training failed: {e}")
        import traceback
        traceback.print_exc()
        return {'error': str(e)}

# Train and evaluate final production model
if optimized_pipeline is not None and best_model_name:
    print(f"\n🏆 Starting Final Production Model Training")
    
    # Ensure optimal_threshold is available from the threshold optimization step
    # If threshold_results contains an error or optimal_threshold is not set, use a default (e.g., 0.5)
    current_optimal_threshold = threshold_results.get('optimal_threshold', 0.5) if isinstance(threshold_results, dict) else 0.5
    if 'error' in threshold_results:
        print(f"⚠ Using default threshold 0.5 due to error in threshold optimization: {threshold_results['error']}")
        current_optimal_threshold = 0.5
    elif not threshold_results: # If threshold_results is empty
        print(f"⚠ Using default threshold 0.5 as threshold optimization results are empty.")
        current_optimal_threshold = 0.5

    final_results = train_final_production_model(
        pipeline=optimized_pipeline,
        X_train=X_train,
        y_train=y_train,
        X_test=X_test,
        y_test=y_test,
        threshold=current_optimal_threshold, # Corrected: use optimal_threshold (or a safe default)
        model_name=best_model_name,
        results_dir=final_results_dir
    )
    
    if 'error' not in final_results and final_results:
        final_f1_score = final_results.get('optimized_threshold_metrics', {}).get('metrics', {}).get('f1', 0)
        final_precision = final_results.get('optimized_threshold_metrics', {}).get('metrics', {}).get('precision', 0)
        final_recall = final_results.get('optimized_threshold_metrics', {}).get('metrics', {}).get('recall', 0)
        
        print(f"\n🎯 Final Production Model Performance (Threshold: {current_optimal_threshold:.3f}):")
        print(f"   F1-Score: {final_f1_score:.4f}")
        print(f"   Precision: {final_precision:.4f}")
        print(f"   Recall: {final_recall:.4f}")
    elif not final_results:
        print(f"\n⚠ Final model training did not produce results.")
    else: # error in final_results
        print(f"\n⚠ Final model training encountered issues: {final_results.get('error')}")
        # final_results might be {'error': '...'}, so accessing metrics would fail
else:
    print("⚠ Cannot train final model - missing optimized pipeline or best_model_name")
    final_results = {}

# Comprehensive Final Report

This section generates a comprehensive final report summarizing all phases of the DMML project and provides recommendations for deployment.

## Generate Comprehensive Final Report

Create a complete summary of the entire DMML project with recommendations and deployment guidelines.

In [ ]:
def generate_comprehensive_final_report(artifacts: Dict[str, Any], optimization_results: Dict[str, Any],
                                      shap_results: Dict[str, Any], perm_results: Dict[str, Any],
                                      threshold_results: Dict[str, Any], learning_results: Dict[str, Any],
                                      validation_results: Dict[str, Any], final_results: Dict[str, Any],
                                      model_name: str, results_dir: str) -> Dict[str, Any]:
    """
    Generate a comprehensive final report for the entire DMML project.
    
    Args:
        artifacts: Loaded artifacts from previous phases
        optimization_results: Bayesian optimization results
        shap_results: SHAP analysis results
        perm_results: Permutation importance results
        threshold_results: Threshold optimization results
        learning_results: Learning curve results
        validation_results: Validation curve results
        final_results: Final model evaluation results
        model_name: Name of the best model
        results_dir: Directory for saving results
        
    Returns:
        Dictionary containing the comprehensive report
    """
    print(f"\n=== Generating Comprehensive Final Report ===")
    
    try:
        # Compile comprehensive report
        report = {
            'project_overview': {
                'title': 'DMML Binary Risk Classification Project - Final Report',
                'objective': 'Develop a robust binary classification model for risk assessment using temporal-spatial data',
                'methodology': 'End-to-end machine learning pipeline with custom transformers, comprehensive evaluation, and advanced optimization',
                'best_model': model_name,
                'completion_date': datetime.now().isoformat(),
                'total_phases': 4
            },
            
            'data_summary': {
                'training_samples': len(artifacts.get('X_train', [])),
                'test_samples': len(artifacts.get('X_test', [])),
                'total_features': len(artifacts.get('feature_names', [])),
                'class_distribution': {
                    'positive_class_rate': float(artifacts.get('y_train', pd.Series()).mean()) if 'y_train' in artifacts else 0.0,
                    'balanced': 'No' if float(artifacts.get('y_train', pd.Series()).mean()) < 0.4 or float(artifacts.get('y_train', pd.Series()).mean()) > 0.6 else 'Yes'
                },
                'temporal_features': ['YEAR', 'MONTH', 'DAY', 'HOUR'],
                'spatial_features': ['Latitude', 'Longitude']
            },
            
            'preprocessing_summary': {
                'stkde_parameters': {
                    'hs_optimal': artifacts.get('hs_optimal', 0),
                    'ht_optimal': artifacts.get('ht_optimal', 0)
                },
                'feature_engineering': ['STKDE intensity calculation', 'Cyclical encoding', 'Risk label transformation'],
                'preprocessing_pipelines': ['General preprocessing', 'Tree-based preprocessing'],
                'data_leakage_prevention': 'Temporal train/test split implemented'
            },
            
            'model_selection_summary': {
                'models_evaluated': len(artifacts.get('model_results', pd.DataFrame())),
                'best_model': model_name,
                'selection_metric': 'F1-Score',
                'cross_validation': 'TimeSeriesSplit or StratifiedKFold',
                'best_cv_score': artifacts.get('best_model_score', 0.0)
            },
            
            'optimization_summary': optimization_results if optimization_results else {
                'method': 'Not performed',
                'reason': 'Bayesian optimization not available or failed'
            },
            
            'interpretability_summary': {
                'shap_analysis': shap_results.get('shap_available', False),
                'permutation_importance': perm_results.get('permutation_available', False),
                'feature_importance_available': shap_results.get('shap_available', False) or perm_results.get('permutation_available', False)
            },
            
            'threshold_optimization_summary': {
                'performed': 'error' not in threshold_results,
                'recommended_threshold': threshold_results.get('recommended_threshold', 0.5),
                'expected_improvement': threshold_results.get('recommended_f1', 0.0) - 0.5 if 'recommended_f1' in threshold_results else 0.0
            },
            
            'learning_analysis_summary': {
                'learning_curves': 'error' not in learning_results,
                'validation_curves': validation_results.get('validation_available', False),
                'overfitting_detected': learning_results.get('overfitting_gap', 0.0) > 0.1,
                'recommendations': learning_results.get('recommendations', [])
            },
            
            'final_performance': final_results.get('optimized_threshold', {}).get('metrics', {}) if final_results else {},
            
            'model_artifacts': {
                'final_model_file': f"{model_name}_final_model.joblib",
                'preprocessing_artifacts': ['preprocessor_general.joblib', 'preprocessor_trees.joblib'],
                'stkde_parameters': 'preprocessing_metadata.json',
                'feature_names': 'feature_names.joblib'
            }
        }
        
        # Generate recommendations
        recommendations = []
        
        # 1. Deployment Recommendations
        recommendations.append("Deploy the model using the saved artifacts and ensure the runtime environment matches the development environment.")
        recommendations.append("Monitor model performance and retrain with new data if performance degrades.")
        
        # 2. Data Management Recommendations
        if artifacts.get('y_train') is not None and len(artifacts['y_train']) > 0:
            positive_rate = artifacts['y_train'].mean()
            if positive_rate < 0.1:
                recommendations.append("Consider using anomaly detection methods due to low positive class rate.")
            elif positive_rate > 0.9:
                recommendations.append("Consider using specialized methods for imbalanced classes.")
        
        # 3. Feature Engineering Recommendations
        if artifacts.get('hs_optimal') and artifacts.get('ht_optimal'):
            recommendations.append(f"HS and HT parameters for STKDE are set to optimal values: HS={artifacts['hs_optimal']}, HT={artifacts['ht_optimal']}.")
        else:
            recommendations.append("Review STKDE parameters (HS, HT) for potential optimization.")
        
        # 4. Model Monitoring Recommendations
        recommendations.append("Implement monitoring for model drift and data quality.")
        recommendations.append("Schedule regular intervals for model evaluation and retraining.")
        
        # 5. Performance Optimization Recommendations
        if final_results.get('optimized_threshold', {}).get('metrics', {}).get('f1', 0) < 0.7:
            recommendations.append("F1 score is below 0.7, consider further tuning or using a different model.")
        if final_results.get('optimized_threshold', {}).get('metrics', {}).get('roc_auc', 0) < 0.7:
            recommendations.append("ROC AUC score is below 0.7, consider improving feature engineering or model selection.")
        
        report['recommendations'] = recommendations
        
        # Save comprehensive report to file
        report_file = os.path.join(results_dir, "comprehensive_final_report.json")
        with open(report_file, 'w') as f:
            json.dump(report, f, indent=2, default=str)
        
        print(f"✓ Comprehensive final report saved to: {report_file}")
        
        return report
    
    except Exception as e:
        print(f"❌ Failed to generate comprehensive report: {e}")
        return {'error': str(e)}

# Generate comprehensive final report
if best_model_name and final_results:
    print(f"\n📊 Generating Comprehensive Final Report")
    
    comprehensive_report = generate_comprehensive_final_report(
        artifacts=artifacts,
        optimization_results=optimization_results,
        shap_results=shap_results,
        perm_results=perm_results,
        threshold_results=threshold_results,
        learning_results=learning_results,
        validation_results=validation_results,
        final_results=final_results,
        model_name=best_model_name,
        results_dir=final_results_dir
    )
    
    if 'error' not in comprehensive_report:
        print(f"\n🏆 DMML Project Completion Summary:")
        print(f"   ✅ All 4 phases completed successfully")
        print(f"   ✅ Best model: {best_model_name}")
        print(f"   ✅ Final performance achieved")
        print(f"   ✅ Model artifacts saved and ready for deployment")
        print(f"   ✅ Comprehensive documentation generated")
    else:
        print(f"\n⚠ Report generation encountered issues")
else:
    print("⚠ Cannot generate final report - missing required results")
    comprehensive_report = {}

print(f"\n🎉 Phase 4: Advanced Tuning and Final Training - COMPLETED!")
print(f"🎉 DMML Binary Risk Classification Project - FULLY COMPLETED!")